In [7]:
import os
os.environ["NUMBA_NUM_THREADS"] = "20"

import netket as nk
import netket.experimental as nkx
import netket.nn as nknn

from math import pi

from netket.experimental.operator.fermion import destroy as c
from netket.experimental.operator.fermion import create as cdag
from netket.experimental.operator.fermion import number as nc

from vmc_torch.fermion_utils import generate_random_fpeps
import quimb.tensor as qtn
import symmray as sr
import pickle

# Define the lattice shape
L = 4  # Side of the square
Lx = int(L)
Ly = int(L//2)
# graph = nk.graph.Square(L)
graph = nk.graph.Grid([Lx,Ly], pbc=False)
N = graph.n_nodes


# Define the fermion filling and the Hilbert space
N_f = int(Lx*Ly/2)-2
hi = nkx.hilbert.SpinOrbitalFermions(N, s=None, n_fermions=N_f)


# Define the Hubbard Hamiltonian
t = 1.0
V = 4.0
H = 0.0
for (i, j) in graph.edges(): # Definition of the Hubbard Hamiltonian
    # print(i,j)
    H -= t * (cdag(hi,i) * c(hi,j) + cdag(hi,j) * c(hi,i))
    H += V * nc(hi,i) * nc(hi,j)


# Exact diagonalization of the Hamiltonian for benchmark
sp_h = H.to_sparse() # Convert the Hamiltonian to a sparse matrix
from scipy.sparse.linalg import eigsh
eig_vals, eig_vecs = eigsh(sp_h, k=2, which="SA")
E_gs = eig_vals[0]
print("Exact ground state energy per site:", E_gs/N, 'Total energy:', E_gs)
# H.to_dense(), hi.all_states()

Exact ground state energy per site: -0.4906657300073222 Total energy: -3.9253258400585778


In [42]:
import random
random.choices(graph.edges(), k=10)

[(6, 7),
 (4, 6),
 (0, 1),
 (4, 6),
 (1, 3),
 (0, 2),
 (0, 1),
 (4, 5),
 (2, 3),
 (2, 3)]

In [2]:
# SU in quimb
D = 6
seed = 2
symmetry = 'U1'
peps, parity_config = generate_random_fpeps(Lx, Ly, D, seed, symmetry, Nf=N_f)

edges = qtn.edges_2d_square(Lx, Ly)
site_info = sr.utils.parse_edges_to_site_info(
    edges,
    D,
    phys_dim=2,
    site_ind_id="k{},{}",
    site_tag_id="I{},{}",
)

t = 1.0
V = 4.0
mu = 0.0

terms = {
    (sitea, siteb): sr.fermi_hubbard_spinless_local_array(
        t=t, V=V, mu=mu,
        symmetry=symmetry,
        coordinations=(
            site_info[sitea]['coordination'],
            site_info[siteb]['coordination'],
        ),
    ).fuse((0, 1), (2, 3))
    for (sitea, siteb) in peps.gen_bond_coos()
}
ham = qtn.LocalHam2D(Lx, Ly, terms)
su = qtn.SimpleUpdateGen(peps, ham, compute_energy_per_site=True,D=D, compute_energy_opts={"max_distance":2}, gate_opts={'cutoff':1e-12})

# cluster energies may not be accuracte yet
su.evolve(50, tau=0.3)
# su.evolve(50, tau=0.1)
# su.evolve(100, tau=0.03)
# su.evolve(100, tau=0.01)
# su.evolve(100, tau=0.003)

peps = su.get_state()
peps.equalize_norms_(value=1)


 96%|#########6| 48/50 [00:01<00:00, 34.60it/s]/home/sijingdu/anaconda3/envs/vmc_torch/lib/python3.9/site-packages/cotengra/hyperoptimizers/hyper.py:54: UserWarning: Couldn't find `optuna`, `baytune (btb)`, `chocolate`, `nevergrad` or `skopt` so will use completely random sampling in place of hyper-optimization.
  warnings.warn(
n=50, tau=0.3000, energy~-0.441221: 100%|##########| 50/50 [00:01<00:00, 29.96it/s]


fPEPS(tensors=8, indices=18, Lx=4, Ly=2, max_bond=6)

In [3]:
# Netket Hamiltonian energy
import numpy as np
all_configs = hi.all_states()
psi_vec = np.array([peps.get_amp(config).contract() for config in all_configs])
psi_vec /= np.linalg.norm(psi_vec)
hamiltonian = H.to_dense()
print((psi_vec.conj().T @ (hamiltonian @ psi_vec))/N)

-0.47614223718249304


In [4]:
psi_vec

array([0.05879026, 0.02049499, 0.05575058, 0.14794209, 0.03242654,
       0.0293141 , 0.18423813, 0.220316  , 0.13286498, 0.18452082,
       0.27863956, 0.26117225, 0.2290455 , 0.16333963, 0.04139211,
       0.2376689 , 0.27134258, 0.25155818, 0.27230367, 0.02388555,
       0.05708314, 0.27881706, 0.2921187 , 0.29461132, 0.26990038,
       0.11423202, 0.02245557, 0.02621847])

In [5]:
# Quimb energy
peps.compute_local_expectation(
    ham.terms, normalized=True, max_bond=64,
)/N

np.float64(-0.4761422371824928)

In [ ]:

# save the state
params, skeleton = qtn.pack(peps)

import os
os.makedirs(f'../data/{Lx}x{Ly}/t={t}_V={V}/N={N_f}/{symmetry}/D={D}', exist_ok=True)

with open(f'../data/{Lx}x{Ly}/t={t}_V={V}/N={N_f}/{symmetry}/D={D}/peps_skeleton.pkl', 'wb') as f:
    pickle.dump(skeleton, f)
with open(f'../data/{Lx}x{Ly}/t={t}_V={V}/N={N_f}/{symmetry}/D={D}/peps_su_params.pkl', 'wb') as f:
    pickle.dump(params, f)
    


